# Gesture Recognition

A home electronics company which manufactures state of the art smart televisions. You want to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command that are as followed:

Gesture | Action
--------|--------
Thumbs up |  Increase the volume
Thumbs down | Decrease the volume
Left swipe | 'Jump' backwards 10 seconds
Right swipe | 'Jump' forward 10 seconds  
Stop | Pause the movie
<br>
<br>

### Objectives:<br>
1. **Generator:** The generator should be able to take a batch of videos as input without any error. Steps like cropping, resizing and normalization should be performed successfully. <br>

2. **Model:** Develop a model that is able to train without any errors which will be judged on the total number of parameters (as the inference(prediction) time should be less) and the accuracy achieved. As suggested by Snehansu, start training on a small amount of data and then proceed further. <br>

3. **Write up:** This should contain the detailed procedure followed in choosing the final model. The write up should start with the reason for choosing the base model, then highlight the reasons and metrics taken into consideration to modify and experiment to arrive at the final model.<br>

### Importing Packages and setting up seed value

In [1]:
import numpy as np
import os
import datetime
import os
import matplotlib.pyplot as plt
from imageio import imread
from skimage import io, transform
from skimage.transform import resize

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

#### Initialize path and vairables

In [3]:
train_doc = np.random.permutation(open('../input/gesture-recognition/train.csv').readlines())
val_doc = np.random.permutation(open('../input/gesture-recognition/val.csv').readlines())

# Batch size handled in class

batch_size = 30
img_size_ht = 120
img_size_wd = 120

### Defining Generator Function

In [4]:
def generator(source_path, folder_list, batch_size, factor_ht=0, factor_wd=0 ):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    img_idx = np.arange(0,30,2) #create a list of image numbers you want to use for a particular video
    
    while True:
        t = np.random.permutation(folder_list)


        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),img_size_ht,img_size_wd,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                   
                    # CROP 
                    if factor_ht!=0 or factor_wd!=0:
                        image_ht, image_wd, _ = image.shape
                        ht1 = int(factor_ht / 2)
                        ht2 = image_ht - int(factor_ht /2)
                        wd1 = int(factor_wd / 2)
                        wd2 = image_wd - int(factor_wd / 2)
                        image=image[ht1:ht2, wd1:wd2,:]
                    
                    # Resize and Normalize
                    image = resize(image,(img_size_ht,img_size_wd))
                    image = (image - np.percentile(image,5))/ (np.percentile(image,95) - np.percentile(image,5))
                    batch_data[folder,idx,:,:,0] = image[:,:,0]
                    batch_data[folder,idx,:,:,1] = image[:,:,1]
                    batch_data[folder,idx,:,:,2] = image[:,:,2]
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        rem_image = len(folder_list)%batch_size
        batch += 1
        if(rem_image!=0):
            batch_data = np.zeros((rem_image,len(img_idx),img_size_ht,img_size_wd,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((rem_image,5)) # batch_labels is the one hot representation of the output
            for folder in range(rem_image): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                   # CROP 
                    if factor_ht!=0 or factor_wd!=0:
                        image_ht, image_wd, _ = image.shape
                        ht1 = int(factor_ht / 2)
                        ht2 = image_ht - int(factor_ht /2)
                        wd1 = int(factor_wd / 2)
                        wd2 = image_wd - int(factor_wd / 2)
                        image=image[ht1:ht2, wd1:wd2,:]
                    
                    # Resize and Normalize
                    image = resize(image,(img_size_ht,img_size_wd))
                    image = (image - np.percentile(image,5))/ (np.percentile(image,95) - np.percentile(image,5))
                    batch_data[folder,idx,:,:,0] = image[:,:,0]
                    batch_data[folder,idx,:,:,1] = image[:,:,1]
                    batch_data[folder,idx,:,:,2] = image[:,:,2]
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
            

In [5]:
def generator_new(source_path, folder_list, batch_size, factor_ht=0, factor_wd=0 ):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    img_idx = np.arange(0,30,2) #create a list of image numbers you want to use for a particular video
    
    while True:
        t = np.random.permutation(folder_list)


        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),img_size_ht,img_size_wd,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                   
                    # CROP 
                    if factor_ht!=0 or factor_wd!=0:
                        image_ht, image_wd, _ = image.shape
                        ht1 = int(factor_ht / 2)
                        ht2 = image_ht - int(factor_ht /2)
                        wd1 = int(factor_wd / 2)
                        wd2 = image_wd - int(factor_wd / 2)
                        image=image[ht1:ht2, wd1:wd2,:]
                    
                    # Resize and Normalize
                    image = resize(image,(img_size_ht,img_size_wd))
#                     image = (image - np.percentile(image,5))/ (np.percentile(image,95) - np.percentile(image,5))
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        rem_image = len(folder_list)%batch_size
        batch += 1
        if(rem_image!=0):
            batch_data = np.zeros((rem_image,len(img_idx),img_size_ht,img_size_wd,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((rem_image,5)) # batch_labels is the one hot representation of the output
            for folder in range(rem_image): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                   # CROP 
                    if factor_ht!=0 or factor_wd!=0:
                        image_ht, image_wd, _ = image.shape
                        ht1 = int(factor_ht / 2)
                        ht2 = image_ht - int(factor_ht /2)
                        wd1 = int(factor_wd / 2)
                        wd2 = image_wd - int(factor_wd / 2)
                        image=image[ht1:ht2, wd1:wd2,:]
                    
                    # Resize and Normalize
                    image = resize(image,(img_size_ht,img_size_wd))
#                     image = (image - np.percentile(image,5))/ (np.percentile(image,95) - np.percentile(image,5))
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
            

In [6]:
def generator(source_path, folder_list, batch_size, factor_ht=0, factor_wd=0 ):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    img_idx = np.arange(0,30,2) #create a list of image numbers you want to use for a particular video
    
    while True:
        t = np.random.permutation(folder_list)


        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),img_size_ht,img_size_wd,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                   
                    # CROP 
                    if factor_ht!=0 or factor_wd!=0:
                        image_ht, image_wd, _ = image.shape
                        ht1 = int(factor_ht / 2)
                        ht2 = image_ht - int(factor_ht /2)
                        wd1 = int(factor_wd / 2)
                        wd2 = image_wd - int(factor_wd / 2)
                        image=image[ht1:ht2, wd1:wd2,:]
                    
                    # Resize and Normalize
                    image = resize(image,(img_size_ht,img_size_wd))
                    image = (image - np.percentile(image,5))/ (np.percentile(image,95) - np.percentile(image,5))
                    batch_data[folder,idx,:,:,0] = image[:,:,0]
                    batch_data[folder,idx,:,:,1] = image[:,:,1]
                    batch_data[folder,idx,:,:,2] = image[:,:,2]
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        rem_image = len(folder_list)%batch_size
        batch += 1
        if(rem_image!=0):
            batch_data = np.zeros((rem_image,len(img_idx),img_size_ht,img_size_wd,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((rem_image,5)) # batch_labels is the one hot representation of the output
            for folder in range(rem_image): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                   # CROP 
                    if factor_ht!=0 or factor_wd!=0:
                        image_ht, image_wd, _ = image.shape
                        ht1 = int(factor_ht / 2)
                        ht2 = image_ht - int(factor_ht /2)
                        wd1 = int(factor_wd / 2)
                        wd2 = image_wd - int(factor_wd / 2)
                        image=image[ht1:ht2, wd1:wd2,:]
                    
                    # Resize and Normalize
                    image = resize(image,(img_size_ht,img_size_wd))
                    image = (image - np.percentile(image,5))/ (np.percentile(image,95) - np.percentile(image,5))
                    batch_data[folder,idx,:,:,0] = image[:,:,0]
                    batch_data[folder,idx,:,:,1] = image[:,:,1]
                    batch_data[folder,idx,:,:,2] = image[:,:,2]
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
            

#### Defining Fit function for the models

In [7]:
def run_model(model, batch_size = 30, epoch_num=25):
        
        # Let us create the train_generator and the val_generator which will be used in .fit_generator.
        train_generator = generator(train_path, train_doc, batch_size)
        val_generator = generator(val_path, val_doc, batch_size)
        

        model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
        if not os.path.exists(model_name):
            os.mkdir(model_name)

        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

        LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=5) # write the REducelronplateau code here
        
        callbacks_list = [checkpoint, LR]
        
        
        # Let us create the train_generator and the val_generator which will be used in .fit_generator.
        if (num_train_sequences%batch_size) == 0:
            steps_per_epoch = int(num_train_sequences/batch_size)
        else:
            steps_per_epoch = (num_train_sequences//batch_size) + 1

        if (num_val_sequences%batch_size) == 0:
            validation_steps = int(num_val_sequences/batch_size)
        else:
            validation_steps = (num_val_sequences//batch_size) + 1
       
        # Let us now fit the model. This will start training the model and with the help of the checkpoints, 
        # you'll be able to save the model at the end of each epoch.
        history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=epoch_num, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
        
        return history

In [8]:
def run_model_new(model, batch_size = 30, epoch_num=25):
        
        # Let us create the train_generator and the val_generator which will be used in .fit_generator.
        train_generator = generator_new(train_path, train_doc, batch_size)
        val_generator = generator_new(val_path, val_doc, batch_size)
        

        model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
        if not os.path.exists(model_name):
            os.mkdir(model_name)

        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

        LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=5) # write the REducelronplateau code here
        
        callbacks_list = [checkpoint, LR]
        
        
        # Let us create the train_generator and the val_generator which will be used in .fit_generator.
        if (num_train_sequences%batch_size) == 0:
            steps_per_epoch = int(num_train_sequences/batch_size)
        else:
            steps_per_epoch = (num_train_sequences//batch_size) + 1

        if (num_val_sequences%batch_size) == 0:
            validation_steps = int(num_val_sequences/batch_size)
        else:
            validation_steps = (num_val_sequences//batch_size) + 1
       
        # Let us now fit the model. This will start training the model and with the help of the checkpoints, 
        # you'll be able to save the model at the end of each epoch.
        history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=epoch_num, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
        
        return history

#### Defining Loss function that will be used to plot the graph

In [9]:
def loss(history):
        acc = history.history['categorical_accuracy']
        val_acc = history.history['val_categorical_accuracy']

        loss = history.history['loss']
        val_loss = history.history['val_loss']

        epochs_range = range(len(history.history['loss']))

        plt.figure(figsize=(8, 8))
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, acc, label='Training Accuracy')
        plt.plot(epochs_range, val_acc, label='Validation Accuracy')
        plt.legend(loc='lower right')
        plt.title('Training and Validation Accuracy')

        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, loss, label='Training Loss')
        plt.plot(epochs_range, val_loss, label='Validation Loss')
        plt.legend(loc='upper right')
        plt.title('Training and Validation Loss')
        plt.show()

**More parameters**

In [10]:
curr_dt_time = datetime.datetime.now()
train_path = '../input/gesture-recognition/train'
val_path = '../input/gesture-recognition/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 25 # choose the number of epochs
print ('# epochs =', num_epochs)
num_classes = 5

### Testing Generator function

In [11]:
# Test Generator Function

xtrain=generator(train_path,train_doc, 32 )
x=next(xtrain)
plt.imshow(x[0][1][1])

In [12]:
# Test Crop in Generator Function

xtrain=generator(train_path,train_doc, 32, factor_ht=10, factor_wd=10)
x=next(xtrain)
plt.imshow(x[0][1][1])

### Define Keras Classes

In [13]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, LeakyReLU
from tensorflow.keras.applications import EfficientNetB0

## Developing Models

### Model 1 

- Vanilla Model

In [14]:
#write your model here
Input_shape = (15, img_size_ht,img_size_wd, 3)
model_1 = Sequential()
model_1.add(Conv3D(8, (3,3,3), padding='same',
                 input_shape=Input_shape))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(Conv3D(16, (3, 3,3)))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2, 2,2)))

model_1.add(Dropout(0.5))

model_1.add(Conv3D(32, (3, 3,3), padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(Conv3D(64, (3, 3,3)))
model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D(pool_size=(2, 2,2)))

model_1.add(Dropout(0.5))

model_1.add(Flatten())

model_1.add(Dense(5))
model_1.add(Activation('softmax'))



In [15]:
#write your model here
Input_shape = (15, img_size_ht,img_size_wd, 3)

lrelu = lambda x: LeakyReLU(alpha=0.1)(x)

model_1a = Sequential()
model_1a.add(Conv3D(8, (3,3,3), padding='same',
                 input_shape=Input_shape))
model_1a.add(BatchNormalization())
model_1a.add(Activation(lrelu))

model_1a.add(Conv3D(16, (3, 3,3)))
model_1a.add(BatchNormalization())
model_1a.add(Activation(lrelu))
model_1a.add(MaxPooling3D(pool_size=(2, 2,2)))

model_1a.add(Dropout(0.5))

model_1a.add(Conv3D(32, (3, 3,3), padding='same'))
model_1a.add(BatchNormalization())
model_1a.add(Activation(lrelu))

model_1a.add(Conv3D(64, (3, 3,3)))
model_1a.add(Activation(lrelu))
model_1a.add(BatchNormalization())
model_1a.add(MaxPooling3D(pool_size=(2, 2,2)))

model_1a.add(Dropout(0.5))

model_1a.add(Flatten())

model_1a.add(Dense(5))
model_1a.add(Activation('softmax'))



In [20]:
optimiser = Adam(learning_rate=0.001) 
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())
history= run_model(model_1)
loss(history)

In [21]:
optimiser = RMSprop(learning_rate=0.001) 
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())
history= run_model(model_1)
loss(history)

In [23]:
optimiser = Adam(learning_rate=0.001) 
model_1a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1a.summary())
history= run_model(model_1a)
loss(history)

In [24]:
optimiser = RMSprop(learning_rate=0.001) 
model_1a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1a.summary())
history= run_model(model_1a)
loss(history)

In [16]:

optimiser = Adam(learning_rate=0.001) 
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())
history= run_model_new(model_1)
loss(history)
# add Codeadd Markdown



In [31]:
optimiser = RMSprop(learning_rate=0.001) 
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())
history= run_model_new(model_1)
loss(history)



In [32]:
optimiser = Adam(learning_rate=0.001) 
model_1a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1a.summary())
history= run_model_new(model_1a)
loss(history)


In [33]:

optimiser = RMSprop(learning_rate=0.001) 
model_1a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1a.summary())
history= run_model_new(model_1a)
loss(history)


# GRU

In [34]:
model_7 = Sequential()

model_7.add(TimeDistributed(Conv2D(16, (3,3), padding = "same"), input_shape=(15,img_size_ht,img_size_wd,3)))
model_7.add(TimeDistributed(BatchNormalization()))
model_7.add(TimeDistributed(Activation('relu')))
           
model_7.add(TimeDistributed(Conv2D(32,(3,3),padding = "same")))
model_7.add(TimeDistributed(BatchNormalization()))
model_7.add(TimeDistributed(Activation('relu')))
model_7.add(TimeDistributed(MaxPooling2D(pool_size= (2,2))))

model_7.add(TimeDistributed(Conv2D(64, (3,3),  padding = "same")))
model_7.add(TimeDistributed(BatchNormalization()))
model_7.add(TimeDistributed(Activation('relu')))
model_7.add(TimeDistributed(MaxPooling2D(pool_size= (2,2))))
           
model_7.add(TimeDistributed(Conv2D(128,(3,3), padding = "same")))
model_7.add(TimeDistributed(BatchNormalization()))
model_7.add(TimeDistributed(Activation('relu')))       
model_7.add(TimeDistributed(MaxPooling2D(pool_size= (2,2))))

model_7.add(TimeDistributed(Conv2D(256,(3,3), padding = "same")))
model_7.add(TimeDistributed(BatchNormalization()))
model_7.add(TimeDistributed(Activation('relu')))          
model_7.add(TimeDistributed(MaxPooling2D(pool_size= (2,2))))


model_7.add(TimeDistributed(Flatten()))
model_7.add(GRU(256,dropout=0.2))
model_7.add(Dense(256, activation='relu'))
model_7.add(Dropout(0.2))

model_7.add(Dense(5, activation='softmax'))

In [16]:
#write your model here
Input_shape = (15, img_size_ht,img_size_wd, 3)

lrelu = lambda x: LeakyReLU(alpha=0.1)(x)

model_7a = Sequential()

model_7a.add(TimeDistributed(Conv2D(16, (3,3), padding = "same"), input_shape=(15,img_size_ht,img_size_wd,3)))
model_7a.add(TimeDistributed(BatchNormalization()))
model_7a.add(TimeDistributed(Activation(lrelu)))
           
model_7a.add(TimeDistributed(Conv2D(32,(3,3),padding = "same")))
model_7a.add(TimeDistributed(BatchNormalization()))
model_7a.add(TimeDistributed(Activation(lrelu)))
model_7a.add(TimeDistributed(MaxPooling2D(pool_size= (2,2))))

model_7a.add(TimeDistributed(Conv2D(64, (3,3),  padding = "same")))
model_7a.add(TimeDistributed(BatchNormalization()))
model_7a.add(TimeDistributed(Activation(lrelu)))
model_7a.add(TimeDistributed(MaxPooling2D(pool_size= (2,2))))
           
model_7a.add(TimeDistributed(Conv2D(128,(3,3), padding = "same")))
model_7a.add(TimeDistributed(BatchNormalization()))
model_7a.add(TimeDistributed(Activation(lrelu)))       
model_7a.add(TimeDistributed(MaxPooling2D(pool_size= (2,2))))

model_7a.add(TimeDistributed(Conv2D(256,(3,3), padding = "same")))
model_7a.add(TimeDistributed(BatchNormalization()))
model_7a.add(TimeDistributed(Activation(lrelu)))          
model_7a.add(TimeDistributed(MaxPooling2D(pool_size= (2,2))))


model_7a.add(TimeDistributed(Flatten()))
model_7a.add(GRU(256,dropout=0.2))
model_7a.add(Dense(256, activation=lrelu))
model_7a.add(Dropout(0.2))

model_7a.add(Dense(5, activation='softmax'))



In [36]:
optimiser = Adam(learning_rate=0.001) 
model_7.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7.summary())
history= run_model(model_7)
loss(history)

In [37]:
optimiser = RMSprop(learning_rate=0.001) 
model_7.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7.summary())
history= run_model(model_7)
loss(history)

In [38]:
optimiser = Adam(learning_rate=0.001) 
model_7a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7a.summary())
history= run_model(model_7a)
loss(history)

In [39]:
optimiser = RMSprop(learning_rate=0.001) 
model_7a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7a.summary())
history= run_model(model_7a)
loss(history)

In [40]:

optimiser = Adam(learning_rate=0.001) 
model_7.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7.summary())
history= run_model_new(model_7)
loss(history)
# add Codeadd Markdown



In [41]:
optimiser = RMSprop(learning_rate=0.001) 
model_7.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7.summary())
history= run_model_new(model_7)
loss(history)



In [42]:
optimiser = Adam(learning_rate=0.001) 
model_7a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7a.summary())
history= run_model_new(model_7a)
loss(history)


In [43]:

optimiser = RMSprop(learning_rate=0.001) 
model_7a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7a.summary())
history= run_model_new(model_7a)
loss(history)


In [17]:
optimiser = Adam(learning_rate=0.001) 
model_7a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7a.summary())
history= run_model(model_7a)
loss(history) 

In [18]:
optimiser = RMSprop(learning_rate=0.001) 
model_7a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7a.summary())
history= run_model(model_7a)
loss(history)

In [19]:
optimiser = Adam(learning_rate=0.001) 
model_7a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7a.summary())
history= run_model_new(model_7a)
loss(history)


In [20]:
optimiser = RMSprop(learning_rate=0.001) 
model_7a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7a.summary())
history= run_model_new(model_7a)
loss(history)


In [21]:
import gc

In [23]:
gc.collect()

In [24]:
%load_ext autoreload
%autoreload 2

# setup backend for matplotlibs plots
%matplotlib inline

This is Complete Overfit. But seeing the ipward trend in accracy and Validation Loss, running the model for more epoch we could see improvement in Validation parameters.